Author: Chris J. Dallmann \
Affiliation: University of Wuerzburg \
Last revision: 26-May-2025

In [109]:
from simulation_model import default_params as params
import pandas as pd
import numpy as np
import os
import yaml
import operator # For fast sorting
import sys
sys.path.append(os.getcwd()) # Enable relative paths

In [110]:
# Load simulation data 
experiment_name = "DNg12"
neurons_excited =  [32815, 31635, 32742, 31078]
freq = 150

path_to_completeness = "../data/simulation/completeness_manc_v1.csv"
path_to_class = "../data/simulation/classifications_manc_v1.csv"
path_to_parquet = "../data/simulation/" + experiment_name + "/" + str(freq) + "Hz.parquet"
path_to_csv = "../data/simulation/" + experiment_name + "/" + str(freq) + "Hz.csv"
t_run = 1 # Duration of simulation in seconds, relevant for spike frequency calculation below

df_simulation = pd.read_parquet(path_to_parquet)

In [111]:
# Remove connectome IDs that are not in the completeness.csv
df_completeness = pd.read_csv(path_to_completeness)
neurons_in_model = df_completeness['Unnamed: 0'].tolist()
neurons_excited = [(i) for i in neurons_excited if i in neurons_in_model]
print(len(neurons_excited))

4


In [112]:
# Load connectome annotations
df_class = pd.read_csv(path_to_class)

In [113]:
# Set ranks for sorting
class_rank = {}
class_rank['descending neuron'] = 0
class_rank['intrinsic neuron'] = 1
class_rank['motor neuron'] = 2
class_rank['ascending neuron'] = 3
class_rank['sensory neuron'] = 4
class_rank['efferent neuron'] = 5
class_rank['sensory ascending'] = 6
class_rank['efferent ascending'] = 7

In [114]:
# Create dictionary with simulation data and metadata  
dict_simulation = {}
for neuron in df_simulation["connectome_id"].unique():
    dict_simulation[neuron] = {}
    dict_simulation[neuron]["connectome_id"] = neuron
    
    # Get simulation data 
    df_simulation_neuron = df_simulation[df_simulation["connectome_id"] == neuron]
    trials = df_simulation_neuron.trial.unique().tolist()
    spike_frequency = []
    for trial in trials:
        spikes = df_simulation_neuron[df_simulation_neuron["trial"] == trial]["t"].values
        spike_frequency.append(len(spikes)/t_run)

    dict_simulation[neuron]["spike_frequency"] = np.mean(spike_frequency)
    dict_simulation[neuron]["first_spike"] = spikes[0]

    # MANC
    dict_simulation[neuron]["class"] = df_class["class"][df_class.bodyId == neuron].item()
    dict_simulation[neuron]["predictedNt"] = df_class.predictedNt[df_class.bodyId == neuron].item()
    dict_simulation[neuron]["predictedNtProb"] = df_class.predictedNtProb[df_class.bodyId == neuron].item()
    dict_simulation[neuron]["instance"] = df_class.instance[df_class.bodyId == neuron].item()
    dict_simulation[neuron]["subclass"] = df_class.subclass[df_class.bodyId == neuron].item()
    dict_simulation[neuron]["class_rank"] = class_rank[df_class["class"][df_class.bodyId == neuron].item()]

    if neuron in neurons_excited:
        dict_simulation[neuron]["excited"] = 1
    else:
        dict_simulation[neuron]["excited"] = 0

In [115]:
# Sort dictionary based on spike rate within super class  

# Create list of tuples with relevant sorting information (ID, super class, spike frequency)
sort_list = [(i, dict_simulation[i]["class_rank"], dict_simulation[i]["spike_frequency"]) for i in dict_simulation]

# Sort IDs 
sorted_list = sorted(sort_list, key=operator.itemgetter(2), reverse=True)
sorted_list = sorted(sorted_list, key=operator.itemgetter(1), reverse=False)
sorted_ids = [i[0] for i in sorted_list]

# Sort dictionary
dict_simulation_sorted = {i: dict_simulation[i] for i in sorted_ids}

In [116]:
# Convert dictionary to data frame
df_excel = pd.DataFrame.from_dict(data=dict_simulation_sorted, orient='index')#.reset_index()
df_excel.connectome_id = df_excel.connectome_id.astype(str)
df_excel

,connectome_id,spike_frequency,first_spike,class,predictedNt,predictedNtProb,instance,subclass,class_rank,excited
32742,32742,149.633333,0.0061,descending neuron,acetylcholine,0.939540,TBD,fl,0,1
31635,31635,149.400000,0.0019,descending neuron,acetylcholine,0.930143,TBD,fl,0,1
31078,31078,148.700000,0.0064,descending neuron,acetylcholine,0.961086,TBD,fl,0,1
32815,32815,143.933333,0.0067,descending neuron,acetylcholine,0.943722,TBD,fl,0,1
12820,12820,12.033333,0.0258,descending neuron,acetylcholine,0.963664,TBD,xl,0,0
...,...,...,...,...,...,...,...,...,...,...
11410,11410,1.000000,0.0362,ascending neuron,acetylcholine,0.531210,AN07B003_T1_R,BA,3,0
10550,10550,1.000000,0.6417,ascending neuron,gaba,0.887137,AN03B010_T3_L,IA,3,0
11909,11909,1.000000,0.0994,ascending neuron,acetylcholine,0.939532,AN19B017_T1_L,BA,3,0
11013,11013,1.000000,0.0940,ascending neuron,acetylcholine,0.751441,AN27X002_T1_R,BA,3,0


In [117]:
# Save as csv
df_excel.to_csv(path_to_csv, index=False, header=True)
print('Results saved as ' + path_to_csv)

Results saved as ../data/simulation/DNg12/150Hz.csv
